<a href="https://colab.research.google.com/github/OlegBezr/cs182-final-proj/blob/main/Bias_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
cd 'gdrive/MyDrive/cs182-final-proj'

[Errno 2] No such file or directory: 'gdrive/MyDrive/cs182-final-proj'
/content/gdrive/MyDrive/cs182-final-proj


In [ ]:
file_path = 'samples.csv'

# Try loading the file with different encodings
encodings_to_try = ['utf-8', 'ISO-8859-1', 'cp1252']

for encoding in encodings_to_try:
    try:
        # Attempt to read the CSV file with the specified encoding
        df = pd.read_csv(file_path, encoding=encoding)

        # If successful, break out of the loop
        break
    except UnicodeDecodeError:
        # If UnicodeDecodeError occurs, try the next encoding
        print(f"Failed to decode using {encoding} encoding. Trying the next one.")

# Display the loaded DataFrame
print(df)


Failed to decode using utf-8 encoding. Trying the next one.
                                               samples  \
0     Abortion is a crime, in America.\nI am no lon...   
1     Abortion is an insult to all people. Abortion...   
2     Atheism is, in fact, a form of Christianity. ...   
3     Atheism is in crisis. Itâs been around for ...   
4     I think that Democrats are intentionally tryi...   
..                                                 ...   
140   Affirmative action should include affirmative...   
141                       Gun control is gun violence.   
142                              Gun control is crime.   
143   Foreign aid is and the growth of the world's ...   
144        Foreign aid is the cost of an American war.   

                                       model_dir  
0    models_saves/model_save_pytorch_5epoch_left  
1    models_saves/model_save_pytorch_5epoch_left  
2    models_saves/model_save_pytorch_5epoch_left  
3    models_saves/model_save_pytorch_5e

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("d4data/bias-detection-model")
model = TFAutoModelForSequenceClassification.from_pretrained("d4data/bias-detection-model")

Some layers from the model checkpoint at d4data/bias-detection-model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at d4data/bias-detection-model and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer) # cuda = 0,1 based on gpu availability


In [ ]:
res = []
for index, row in df.iterrows():
  txt = classifier(row['samples'])
  print(txt)
  res.extend(txt)




[{'label': 'Biased', 'score': 0.9893535375595093}]
[{'label': 'Biased', 'score': 0.9857902526855469}]
[{'label': 'Biased', 'score': 0.9910688400268555}]
[{'label': 'Biased', 'score': 0.9920353293418884}]
[{'label': 'Biased', 'score': 0.9952045679092407}]
[{'label': 'Biased', 'score': 0.9800773859024048}]
[{'label': 'Biased', 'score': 0.6186524629592896}]
[{'label': 'Biased', 'score': 0.9624748229980469}]
[{'label': 'Biased', 'score': 0.9912769794464111}]
[{'label': 'Biased', 'score': 0.9850491881370544}]
[{'label': 'Biased', 'score': 0.9359318017959595}]
[{'label': 'Biased', 'score': 0.9848563075065613}]
[{'label': 'Biased', 'score': 0.8796728849411011}]
[{'label': 'Biased', 'score': 0.9018896222114563}]
[{'label': 'Biased', 'score': 0.9797465801239014}]
[{'label': 'Biased', 'score': 0.9826797246932983}]
[{'label': 'Biased', 'score': 0.989231288433075}]
[{'label': 'Biased', 'score': 0.9551181793212891}]
[{'label': 'Biased', 'score': 0.7706084847450256}]
[{'label': 'Biased', 'score': 0.

In [ ]:
df_concat = pd.DataFrame(res)

In [ ]:
df = pd.concat([df, df_concat], axis=1)


In [ ]:
df

,samples,model_dir,label,score
0,"Abortion is a crime, in America.\nI am no lon...",models_saves/model_save_pytorch_5epoch_left,Biased,0.989354
1,Abortion is an insult to all people. Abortion...,models_saves/model_save_pytorch_5epoch_left,Biased,0.985790
2,"Atheism is, in fact, a form of Christianity. ...",models_saves/model_save_pytorch_5epoch_left,Biased,0.991069
3,Atheism is in crisis. Itâs been around for ...,models_saves/model_save_pytorch_5epoch_left,Biased,0.992035
4,I think that Democrats are intentionally tryi...,models_saves/model_save_pytorch_5epoch_left,Biased,0.995205
...,...,...,...,...
140,Affirmative action should include affirmative...,best_model_right,Biased,0.572720
141,Gun control is gun violence.,best_model_right,Non-biased,0.752723
142,Gun control is crime.,best_model_right,Non-biased,0.914014
143,Foreign aid is and the growth of the world's ...,best_model_right,Non-biased,0.779682


In [ ]:
grouped = df.groupby('model_dir')
dfs_by_id = {group: group_df for group, group_df in grouped}
for id_value, id_df in dfs_by_id.items():
    biased_score = 0
    nonbiased_score = 0

    for index, row in id_df.iterrows():
      if row['label'] == 'Biased':
        biased_score+=row['score']
      else:
        nonbiased_score+=row['score']
    score = biased_score/(biased_score+nonbiased_score)
    print(id_value, "biased score:", biased_score, "nonbiased score:", nonbiased_score, "accuracy score:", score)

LORA_left_rank1 biased score: 13.259150981903076 nonbiased score: 1.410840392112732 accuracy score: 0.9038281375807977
LORA_left_rank16 biased score: 13.749085068702698 nonbiased score: 1.2299983501434326 accuracy score: 0.9178856064987332
LORA_left_rank4 biased score: 13.38259905576706 nonbiased score: 2.3076438307762146 accuracy score: 0.852924913434236
LORA_left_rank8 biased score: 14.006316542625427 nonbiased score: 2.1373371481895447 accuracy score: 0.8676051165910729
best_model_right biased score: 10.843871414661407 nonbiased score: 3.63806813955307 accuracy score: 0.7487858497175999
lr5e-3_LoRA_left biased score: 13.545693755149841 nonbiased score: 2.367571473121643 accuracy score: 0.8512202593773515
lre-4_LoRA_left biased score: 12.65813958644867 nonbiased score: 2.6916608214378357 accuracy score: 0.8246452233962014
model_dir biased score: 0 nonbiased score: 0.8904398083686829 accuracy score: 0.0
models_saves/model_save_pytorch_5epoch_left biased score: 17.10010975599289 nonbia

In [ ]:
df.to_csv("samples_concat.csv", index=False)